In [13]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

In [14]:
torch.manual_seed(42)
np.random.seed(42)

In [15]:
class PowerPredictionModel(nn.Module):
    def __init__(self, input_size):
        super(PowerPredictionModel, self).__init__()
        
        self.model = nn.Sequential(
            nn.BatchNorm1d(input_size),
            nn.Linear(input_size, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(0.1),
            
            nn.Linear(32, 1)
        )
    
    def forward(self, x):
        return self.model(x)

In [16]:
def prepare_data(data_path):
    df = pd.read_csv(data_path)

    df['radiation_efficiency'] = df['shortwave_radiation_backwards_sfc'] / (df['total_cloud_cover_sfc'].clip(lower=0.1))
    
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    X_train_tensor = torch.FloatTensor(X_train_scaled)
    y_train_tensor = torch.FloatTensor(y_train.values).reshape(-1, 1)
    X_test_tensor = torch.FloatTensor(X_test_scaled)
    y_test_tensor = torch.FloatTensor(y_test.values).reshape(-1, 1)
    
    return X_train_tensor, X_test_tensor, y_train_tensor, y_test_tensor, scaler


In [17]:
def train_model(model, X_train, y_train, X_test, y_test, epochs=1000):
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
    
    best_r2 = -float('inf')
    best_model_state = None
    patience = 100 
    no_improve_count = 0
    min_delta = 0.0001 
    
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()
        
        model.eval()
        with torch.no_grad():
            test_outputs = model(X_test)
            test_r2 = r2_score(y_test.numpy(), test_outputs.numpy())
            
            if test_r2 > (best_r2 + min_delta):
                best_r2 = test_r2
                best_model_state = model.state_dict().copy()
                no_improve_count = 0
            else:
                no_improve_count += 1
            
            if epoch % 100 == 0:
                print(f'Epoch [{epoch}/{epochs}], Loss: {loss.item():.4f}, R2 Score: {test_r2:.4f}')
            
            if no_improve_count >= patience:
                print(f"Early stopping triggered at epoch {epoch}")
                break
    
    model.load_state_dict(best_model_state)
    return model, best_r2


In [18]:
def main():
    X_train, X_test, y_train, y_test, scaler = prepare_data('../data/spg.csv')
    
    model = PowerPredictionModel(input_size=X_train.shape[1])
    trained_model, best_r2 = train_model(model, X_train, y_train, X_test, y_test)
    
    print(f'\nBest R2 Score: {best_r2:.4f}')
    
    model.eval()
    with torch.no_grad():
        test_predictions = model(X_test)
        final_r2 = r2_score(y_test.numpy(), test_predictions.numpy())
        
        for i in range(min(3, len(X_test))):
            pred = test_predictions[i].item()
            actual = y_test[i].item()
            abs_diff = abs(pred - actual)
            
            print(f'\nSample {i+1} Prediction: {pred:.2f}')
            print(f'Actual Value: {actual:.2f}')
            print(f'Absolute Difference: {abs_diff:.2f}')
            torch.save(model.state_dict(), 'solar.pth')
            if actual != 0:
                perc_diff = (abs_diff/actual) * 100
                print(f'Percentage Difference: {perc_diff:.1f}%')
            else:
                if pred == 0:
                    print('Percentage Difference: 0.0%')
                else:
                    print('Percentage Difference: N/A (actual value is 0)')

if __name__ == "__main__":
    main()

Epoch [0/1000], Loss: 11961482.0000, R2 Score: -0.4181
Epoch [100/1000], Loss: 9079203.0000, R2 Score: -0.0586
Epoch [200/1000], Loss: 3944107.0000, R2 Score: 0.5607
Epoch [300/1000], Loss: 3402995.5000, R2 Score: 0.6208
Epoch [400/1000], Loss: 2969397.2500, R2 Score: 0.6731
Epoch [500/1000], Loss: 2414544.0000, R2 Score: 0.7343
Epoch [600/1000], Loss: 1689230.8750, R2 Score: 0.7855
Epoch [700/1000], Loss: 1012284.4375, R2 Score: 0.7296
Early stopping triggered at epoch 715

Best R2 Score: 0.7872

Sample 1 Prediction: 7713.16
Actual Value: 7458.20
Absolute Difference: 254.96
Percentage Difference: 3.4%

Sample 2 Prediction: 0.00
Actual Value: 0.00
Absolute Difference: 0.00
Percentage Difference: N/A (actual value is 0)

Sample 3 Prediction: 0.00
Actual Value: 28.72
Absolute Difference: 28.71
Percentage Difference: 100.0%
